Maxwell solver for PEC bodies (EFIE)
=============================


In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

We consider a perfect conducter $\Omega \subset \mathbb R^3$ and a plane wave $\boldsymbol E_{\mathrm{inc}} $ with Dirichlet trace $\gamma_D \boldsymbol E_{\mathrm{inc}} = -\boldsymbol m \in \boldsymbol H^{-\frac12}\left( \mathrm{div}_\Gamma, \Gamma\right)\,.$ The incoming wave thus induces a scattered electric field $\boldsymbol E$ which propagates into $\Omega^c$. The scattered electric field solves the following boundary value problem: 

$$ \left\{ \begin{array}{rcl l} \mathbf{curl} \, \mathbf{curl}\, \boldsymbol E - \kappa^2 \, \boldsymbol E &=& \boldsymbol 0, \quad &\textnormal{in } \Omega^c \subset \mathbb R^3\,,\\ \gamma_D \,\boldsymbol E &=& \boldsymbol m, \quad & \textnormal{on }\Gamma \\ \left\| \mathbf{curl} \, \boldsymbol E( x) - i\,\omega\,\epsilon \, \boldsymbol E( x)\right\| &=& \mathcal O\left( \displaystyle \frac{1}{\| x\|^2}\right), &\textnormal{for} \; \|x\| \to \infty\,.\end{array} \right. $$ 

and a possible representation reads

$$ \boldsymbol E(x) = \underbrace{\int_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}\sigma_y + \frac{1}{\kappa^2} \nabla \int_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{SL} (\boldsymbol j) } } \,.$$ 

The density $\boldsymbol j$ is determined by the boundary element method, i.e. the numerical solution of the variational formulation 

$$ \begin{array}{c} \forall \, \boldsymbol v\in H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma): \quad \left\langle \,\mathrm{SL} (\boldsymbol j),\, \boldsymbol v \right\rangle_{-\frac12} = \left\langle \boldsymbol m \times \boldsymbol n, \boldsymbol v\right\rangle_{-\frac12} \,. \end{array}$$ 

In the enineering community, the approximation scheme is also known as **method of moments** (MoM) and the resulting equation for $\boldsymbol j$ is called the **EFIE**, the **electric field integral equation**. 

Define the geometry of the perfect conductor $\Omega$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(2)

Next, we create two finite element spaces for $\boldsymbol H^{-\frac12}(\mathrm{curl}_\Gamma, \Gamma)$ and rotate them later to obtain $\boldsymbol H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma)$ conforming shape functions as test and trial spaces. 

In [ ]:
fesHCurl = HCurl(mesh, order=3, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) conform spaces
print ("ndof HCurl = ", fesHCurl.ndof)

Define the incoming plane wave and compute the given boundary data $\boldsymbol m$: 

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )
n = specialcf.normal(3)
m = -Cross(n,E_inc) # -nxE_inc in H(div_Gamma)

In [ ]:
#Draw (m.real*m.real + m.imag*m.imag, mesh, draw_vol=False, order=2);
Draw (m.real, mesh, draw_vol=False, order=2);
Draw (m.imag, mesh, draw_vol=False, order=2);

Compute the right hand side vector:

In [ ]:
rhs = LinearForm( Cross(m,n) * Cross(n,vHCurl.Trace()) *ds(bonus_intorder=3)).Assemble() # <H(curl_Gamma), H(div_Gamma)> 

The discretisation of the above variational formulation leads to a system of linear equations, ie

$$ V\, \mathrm j = \mathrm{rhs}\,,$$

where the single layer operator $V$ is a regular and symmetric matrix.

In [ ]:
j = GridFunction(fesHCurl) # note: we actually solve for coefficients for FE space H(div_Gamma)
pre = BilinearForm(uHCurl.Trace()*vHCurl.Trace()*ds).Assemble().mat.Inverse(freedofs=fesHCurl.FreeDofs()) # to be changed
with TaskManager(): # pajetrace=1000*1000*1000):
    V = MaxwellSingleLayerPotentialOperator(fesHCurl, kappa, intorder=12, leafsize=40, eta=3., eps=1e-4, method="svd", testhmatrix=False)
    CG(mat = V.mat, pre=pre, rhs = rhs.vec, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

In [ ]:
#Draw (j.real*j.real + j.imag*j.imag, mesh, draw_vol=False, order=2);
#Draw (j.real, mesh, draw_vol=False, order=2);
#Draw (j.imag, mesh, draw_vol=False, order=2);

In [ ]:
Draw( j, mesh, draw_vol=False, order=2, min=-3, max=3, animate_complex=True);

For details on convergence rates and low frequency stabilisation of the EFIE look [here](https://publikationen.sulb.uni-saarland.de/bitstream/20.500.11880/26312/1/thesis_weggler_final_6.1.12.pdf).